# Captain's Loc

In [5]:
# Setup

# pip3 install urllib3 # Might not have worked. Hard to say
# pip install urllib3
# pip3 install requests



In [6]:
# Load the sample data - 1 month from the rest endpoint serving JSON of a Pg database

import json, urllib.request, requests
from collections import namedtuple

def getLocations():

    send_url = 'https://geo-pg.captainjustin.space/transactions_rs_time'
    r = requests.get(send_url)
    j = json.loads(r.text)
    return j

sample_locations = getLocations()

# To See what is in there:
print(sample_locations[0])



{'id': '54', 'lon': 27.9517183, 'lat': -26.118670899999998, 'tim': '2018-09-14T10:20:12+00:00', 'amount': 0.0, 'morning': False, 'midday': True, 'evening': False, 'weekend': False}


In [7]:
# Get the distance between two points
from math import sqrt

def getDistance(loc1, loc2):
    dist = sqrt( (loc2['lon'] - loc1['lon'])**2 + (loc2['lat'] - loc1['lat'])**2 )
    return dist

# To see this work
print(getDistance(sample_locations[0], sample_locations[1]))


0.001564830869456087


In [8]:
# Count the number of points within a distance of a point
def getScore(point, allPoints, limitDistance):
    result = 0
    for other in allPoints:
        thisDistance = getDistance(point, other)
        if thisDistance <= limitDistance:
            result += 1
    return result
print("Function getScore defined")

function defined


In [15]:
# Convert anything to json
import json
def getJson(o):
    json_string = json.dumps(topNList)    
    return json_string
print("Function getJson defined")


Function getJson defined


In [16]:
# Get the score of each point

# Decide on accuracy
arbitaryDistanceLimit = 0.0001

# We'll keep scores here in an array here
scores = [] 

# Define the data structure
class Score:
    def __init__(self, total, item):
        self.total = total
        self.item = item
    def __repr__(self):
        return repr((self.total, self.item))


# Get the score for each location
for l in sample_locations:
    total = getScore(l, sample_locations, arbitaryDistanceLimit)
    scores.append(Score(total, l))

sorted_locations = sorted(scores, key=lambda score: score.total)


print("Scores are complete")


Scores are complete


In [17]:
# Get the top n
topN = 10
topNList = []

current = 0
last = len(sorted_locations) - 1
start = last - topN
a = last
while a > start:
    a -= 1
    topNList.append(sorted_locations[a].item)
    


In [18]:
json_string = getJson(topNList)
print(json_string)


[{"id": "13298", "lon": 27.9478033, "lat": -26.0914297, "tim": "2018-08-20T16:59:38+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "9204", "lon": 27.9477917, "lat": -26.0914091, "tim": "2018-08-28T15:28:50+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "807", "lon": 27.9477749, "lat": -26.091455999999997, "tim": "2018-09-12T21:11:52+00:00", "amount": 0.0, "morning": false, "midday": null, "evening": true, "weekend": false}, {"id": "11675", "lon": 27.9478078, "lat": -26.0914391, "tim": "2018-08-23T14:26:18+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "11677", "lon": 27.9478078, "lat": -26.0914391, "tim": "2018-08-23T14:23:07+00:00", "amount": 0.0, "morning": false, "midday": true, "evening": false, "weekend": false}, {"id": "9112", "lon": 27.947792, "lat": -26.0914592, "tim": "2018-08-28T21:58:43+00:00", "amount": 0.0, "morni